In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from tensorflow.keras.models import load_model

import pickle

In [9]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
#Preprocessing
#drop irrelevant cols
data=data.drop(["RowNumber","CustomerId","Surname"],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [12]:
## Encode cat variables
label_encoder_gender=LabelEncoder()
data["Gender"]=label_encoder_gender.fit_transform(data["Gender"])
data


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [13]:
from sklearn.preprocessing import OneHotEncoder

In [15]:
onehot_encoding_geo=OneHotEncoder()
geo_encoder=onehot_encoding_geo.fit_transform(data[['Geography']])

In [18]:
geo_encoder=geo_encoder.toarray()

In [19]:
geo_df=pd.DataFrame(geo_encoder,columns=onehot_encoding_geo.get_feature_names_out())

In [21]:
data=pd.concat([data.drop("Geography",axis=1),geo_df],axis=1)

In [22]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [23]:
with open('Label_encoder1.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('one_hot_encoding_geo1.pkl','wb') as file:
    pickle.dump(onehot_encoding_geo,file)

In [24]:
#dive data into independent and dependent features
X=data.drop('EstimatedSalary',axis=1)
y=data["EstimatedSalary"]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)



In [25]:
with open("scaler1.pkl",'wb') as file:
    pickle.dump(scaler,file)

In [26]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [29]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),#1st hidden layer
    Dense(32,activation='relu'),
    Dense(1)

])

In [30]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

In [33]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [34]:
log_dir='regression/fit'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [35]:
earlystop_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [36]:
hist=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,earlystop_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 640us/step - loss: 101437.5938 - mae: 101437.5938 - val_loss: 98464.2578 - val_mae: 98464.2578
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step - loss: 100525.2656 - mae: 100525.2656 - val_loss: 96695.7109 - val_mae: 96695.7109
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 374us/step - loss: 97496.7891 - mae: 97496.7891 - val_loss: 92257.0547 - val_mae: 92257.0547
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 396us/step - loss: 91596.8750 - mae: 91596.8750 - val_loss: 85015.8203 - val_mae: 85015.8203
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 371us/step - loss: 84417.1484 - mae: 84417.1484 - val_loss: 75883.2188 - val_mae: 75883.2188
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 372us/step - loss: 74675.5469 - mae: 74675.5469 - val_loss: 66766.7422 - val_mae: 66766.7422
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 373us/step - loss: 66340.9609 - mae: 66340.9609 - val_loss: 59290.7109 - val_mae: 59290.7109
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━

In [37]:
model.save('model.h5')

In [38]:
#Load tensorflow extension
%load_ext tensorboard

In [39]:
%tensorboard --logdir regression/fit20240911-162928/ 

In [40]:
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE:{test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 165us/step - loss: 51144.0000 - mae: 51144.0000
Test MAE:50271.359375
